In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../data/process_v1/process_data_4.csv')

In [ ]:
del df['unitPrice']

In [ ]:
del df['landType']

In [ ]:
df.head(3)

In [ ]:
df['w'] = df['frontWidth']
df['h'] = df['landSize'] / df['w']

In [ ]:
df = df.replace(np.inf, np.nan)

In [ ]:
df.columns.tolist()

In [ ]:
location_df = df[['lat', 'lon']].drop_duplicates()

In [ ]:
facility_500_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_500_facility_count.csv')
facility_1000_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_1000_facility_count.csv')
facility_2000_df = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_hn_distance_2000_facility_count.csv')

In [ ]:
def rename_col(dataframe, distance):
    cols = [c for c in dataframe.columns if c  not in ['lat', 'lon']]
    rename_cols = [f'num_of_{col}_in_{distance}m_radius' for col in cols]

    rename_col_dict = dict(zip(cols, rename_cols))
    return dataframe.rename(columns = rename_col_dict)

facility_500_df = rename_col(facility_500_df, 500)
facility_1000_df = rename_col(facility_1000_df, 1000)
facility_2000_df = rename_col(facility_2000_df, 2000)


In [ ]:
for facility_df in [facility_500_df, facility_1000_df, facility_2000_df]:
    df = df.merge(facility_df, how='left', on = ['lat', 'lon'])

In [ ]:
df

In [ ]:
num_cols = []
cat_cols = []
for c in df.columns.tolist():
    if df[c].dtype != 'object':
        num_cols.append(c)
    else:
        cat_cols.append(c)


In [ ]:
cat_cols

In [ ]:
num_cols

In [ ]:
df[
'typeOfRealEstate'
].value_counts()

In [ ]:
certificateOfLandUseRight_list = list(df['certificateOfLandUseRight'].unique())
certificateOfLandUseRight_dict = dict(zip(certificateOfLandUseRight_list, range(len(certificateOfLandUseRight_list))))
df['certificateOfLandUseRight'] = df['certificateOfLandUseRight'].map(certificateOfLandUseRight_dict)

ward_list = list(df['ward'].unique())
ward_dict = dict(zip(ward_list, range(len(ward_list))))
df['ward'] = df['ward'].map(ward_dict)
street_list = list(df['street'].unique())
street_dict = dict(zip(street_list, range(len(street_list))))
df['street'] = df['street'].map(street_dict)
district_list = list(df['district'].unique())
district_dict = dict(zip(district_list, range(len(district_list))))
df['district'] = df['district'].map(district_dict)
city_list = list(df['city'].unique())
city_dict = dict(zip(city_list, range(len(city_list))))
df['city'] = df['city'].map(city_dict)
typeOfRealEstate_list = list(df['typeOfRealEstate'].unique())
typeOfRealEstate_dict = dict(zip(typeOfRealEstate_list, range(len(typeOfRealEstate_list))))
df['typeOfRealEstate'] = df['typeOfRealEstate'].map(typeOfRealEstate_dict)
facade_list = list(df['facade'].unique())
facade_dict = dict(zip(facade_list, range(len(facade_list))))
df['facade'] = df['facade'].map(facade_dict)
houseDirection_list = list(df['houseDirection'].unique())
houseDirection_dict = dict(zip(houseDirection_list, range(len(houseDirection_list))))
df['houseDirection'] = df['houseDirection'].map(houseDirection_dict)
accessibility_list = list(df['accessibility'].unique())
accessibility_dict = dict(zip(accessibility_list, range(len(accessibility_list))))
df['accessibility'] = df['accessibility'].map(accessibility_dict)

In [ ]:
district_dict

In [ ]:
df1 = pd.read_csv('../feature_repo/src/feature_engineering/population/data/table/hcm_population.csv')
df2 = pd.read_csv('../feature_repo/src/feature_engineering/population/data/table/hn_population.csv')

population_df = pd.concat([
    df1, df2
])

def convert_district_to_num(district):
    if district in ['mê linh',
        'ba vì',
        'phúc thọ',
        'thạch thất',
        'mỹ đức',
        'sơn tây',
        'quốc oai',
        'quốc oai']:
        district = 'suburb_west'
    elif district in ['thanh oai',
'ứng hòa',
'phú xuyên',
'thường tín',
'chương mỹ',]:
        district = 'suburb_south'
    elif district in ['sóc sơn', 'đan phượng']:
        district = 'suburb_north'
    candidate_str = [district, f"quận {district}", f"huyện {district}", f"thị xã {district}"]

    for candidate in candidate_str:
        try:
            return district_dict[candidate]
        except:pass
    print(district)
population_df['district'] = population_df['name'].apply(convert_district_to_num)
population_df

In [ ]:
del population_df['average_price']

In [ ]:
population_df = population_df.dropna(subset = ['district'])

In [ ]:
population_df = population_df.sort_values(by = 'district')
population_df

In [ ]:
population_df[['administrative_genre', 'district']]

In [ ]:
ad_df = population_df[['administrative_genre', 'district']].drop_duplicates(subset = ['district'], keep = 'first')
ad_df

In [ ]:
population_df = population_df.groupby('district')[['acreage', 'population', 'density', 'lat', 'lon', 'num_of_ward']].mean().reset_index()

In [ ]:
population_df = ad_df.merge(population_df, how='left', on = ['district'])
population_df

In [ ]:
ad_list = list(population_df['administrative_genre'].unique())
ad_dict = dict(zip(ad_list, range(len(ad_list))))
ad_dict

In [ ]:
population_df['administrative_genre'] = population_df['administrative_genre'].map(ad_dict)
population_df = population_df.rename(columns = {'lat': 'district_lat', 'lon': 'district_lon'})

In [ ]:
df = df.merge(population_df, how='left', on = 'district')
df

In [ ]:
del df['numberOfKitchens']

In [ ]:
del df['numberOfGarages']

In [ ]:
city_dict

In [ ]:
best_df1 = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hcm_famous_place.csv')
best_df1['city'] = 'hồ chí minh'

best_df2 = pd.read_csv('../feature_repo/src/feature_engineering/facility/data/table/hn_famous_place.csv')
best_df2['city'] = 'hà nội'

best_df = pd.concat([best_df1, best_df2])
best_df

In [ ]:
del best_df['address']
best_df['city'] = best_df['city'].map(city_dict)

In [ ]:
import math
from tqdm import tqdm
def distance_func(lat1: float, lon1: float, lat2: float, lon2: float):

    try:
        R = 6371
        dLat = (lat2-lat1) * math.pi / 180
        dLon = (lon2-lon1) * math.pi / 180
        lat1 = lat1 * math.pi / 180
        lat2 = lat2 * math.pi / 180
        a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * \
            math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = R * c
        return d*1000
    except:
        print(f"Distance_Func error with: {(lat1, lon1, lat2, lon2)}")

def distance(lat1, lon1, lat2, lon2, city1, city2):
    if city1 != city2:
        return -1
    return distance_func(lat1, lon1, lat2, lon2)


best_arr = best_df.values
for i in tqdm(range(len(best_arr))):
    lat, lon = best_arr[i][0], best_arr[i][1]
    name = best_arr[i][2]
    city = 'hn' if best_arr[i][3] == 0 else 'hcm'

    df[f'distance_{city}_{name}'] = df.apply(lambda x: distance(x['lat'], x['lon'], lat, lon, x['city'], best_arr[i][3]), axis = 1)


In [ ]:
df['distance_district'] = df.apply(lambda x: distance_func(x['lat'], x['lon'], x['district_lat'], x['district_lon']), axis = 1)

In [ ]:
distance_cols = [c for c in df.columns.tolist() if 'distance' in c]
for c in distance_cols:
    df[c] = np.log(df[c])


In [209]:
df['target'] = df['price'] * 1000 / df['landSize']

/tmp/ipykernel_852678/343422437.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['target'] = df['price'] * 1000 / df['landSize']


In [213]:
df['population'] = np.log(df['population'])

In [215]:
df['density'] = np.log(df['density'])

In [217]:
df['acreage'] = np.log(df['acreage'])

In [220]:
df

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,certificateOfLandUseRight,ward,street,district,city,lat,lon,...,distance_hn_Hàng Khay,distance_hn_Hàng Trống,distance_hn_Hàng Hành,distance_hn_Phố Hai Bà Trưng,distance_hn_Phố Lý Thường Kiệt,distance_hn_Phố Bà Triệu,distance_hn_Hàng Bài,distance_hn_Phố Hai Bà Trưng(Hoàn Kiếm),distance_district,target
0,1,NaN,NaN,0,0,0,0,0,21.036403,105.727235,...,9.450133,9.416401,9.433240,9.402095,9.411188,9.445093,9.435854,9.433688,9.404118,92.000000
1,5,5.0,4.0,1,1,1,1,0,20.985272,105.843605,...,8.423630,8.481227,8.543236,8.423707,8.377326,8.051868,8.301442,8.401147,8.012053,102.857143
2,4,NaN,NaN,1,2,2,2,0,20.978546,105.796775,...,8.931940,8.924143,8.972384,8.879267,8.866793,8.772982,8.861183,8.903859,9.910879,97.500000
3,7,7.0,7.0,1,3,3,3,0,21.028933,105.855453,...,6.598654,6.956718,6.761142,7.156634,7.141550,7.581442,7.109375,6.869687,7.695956,1375.000000
4,4,NaN,NaN,1,4,4,4,0,21.046094,105.800236,...,8.622254,8.530771,8.551876,8.516947,8.551225,8.704269,8.624514,8.592451,8.628126,534.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170959,1,2.0,2.0,0,119,600,9,0,21.001092,105.869070,...,8.144654,8.276105,8.302429,8.257995,8.201712,7.903704,8.078461,8.165627,7.980783,38.571429
170960,6,10.0,10.0,0,231,358,11,0,20.999584,105.817799,...,8.384273,8.369235,8.451888,8.289643,8.267210,8.118311,8.261364,8.334477,7.724700,155.333333
170961,8,22.0,22.0,1,164,574,10,0,21.029128,105.886118,...,8.256108,8.350590,8.297314,8.398680,8.382899,8.404012,8.333913,8.312352,9.010103,304.878049
170962,5,NaN,NaN,1,35,125,15,0,21.017355,105.775199,...,8.953748,8.907498,8.944352,8.875072,8.883976,8.911851,8.916458,8.924393,8.647240,157.971014


In [221]:
df.to_csv('../data/process_v1/process_data_5.csv', index = False)